# Intro aux index

## SQL DB vs Index
Les index sont des cousins des bases de données (DB - databse) dans le sens où ils stockent de la données et permettent d'y accéder via des requêtes. Commençons par voir les index comme des DB munis d'une seule table qui contient des **documents** qu'un utilisateur voudra requêter. Les index ne sont pas capables de jointure et les opérations classiques du SQL y sont difficiles (`GROUP BY`) voire simplement impossibles (`JOIN`, `ROW NUMBER`). Appelons *corpus* l'ensemble de documents indéxés.

NB: dans une moindre mesure, les *index* sont régulièrement utilisés dans les coulisses des DB SQL sur les clefs fréquemment manipulés. On y reviendra.

### Alors à quoi sert un index ?

Un **index** est un puissant outil *pour retrouver des documents à partir de **query** sur leurs attributs, une partie de leurs attributs, ou d'une information partielle sur leurs attributs*. De plus, ces outils sont souvent équipés d'un système de **scores d'adéquation** qui *ordonnent* les documents par ordre de pertinence face à la query.

Il existe une infinité de façon de définir un score d'adéquation. En effet, il ne s'agit "que" d'un calcul opéré sur le coupe `(query, document)` et qui propose une mesure de l'adéquation query/document. Nous parlerons simplement de *score* par la suite

# Index lexical

## Structure de donné de l'index lexical inversé

Un index est capable de retrouver extrêmement vite des attributs d'un documents à partir d'une requête. Les index sont équipés de structures de données complexes permettant d'insérer des documents et d'y effectuer des recherches en complexité $O(\log n)$ (où $n$ est le nombre de documents). Contrairement à une `hashMap`, il ne s'agit pas de simplement `get` un document via son identifiant (auquel cas il s'agit d'une compléxité de $O(1)$) mais de trouver *tous les documents qui correspondent à une recherche*. 

Exemple : trouver tous les documents qui contiennent le mot `pils`. 

Un index maintient une ["posting list" ou "index inversé"](https://www.wikiwand.com/en/articles/Inverted_index) en mémoire : à la façon d'un glossaire dans un livre, l'index maintient en mémoire un vocabulaire (les mots contenus dans tout le corpus) ainsi que, pour chaque mot, **la liste de tous les documents qui l'évoquent ainsi que les positions du mot dans les documents**. Trouver les documents qui contiennent une série de mot devient rapide : $O(\log n)$.

Exemple :

Soit le (mini) corpus suivant
- "portez ce vieux whiskey au juge blond qui fume"
- "On fume ce type de malts pour obtenir ce whiskey tourbé"

Calculons la `posting_list`

In [60]:
corpus = [
    "portez ce vieux whiskey au juge blond qui fume",
    "On fume ce malt pour obtenir ce whiskey tourbé",
    "La fermentation haute donne des bières plus fruitées.", 
    "Ce houblon ajoute une amertume distincte à la bière.", 
    "Le malt apporte la rondeur à la bière mais un malt torréfié apporte des arômes de café à la bière.", 
    "La bière de fermentation basse est plus légère et rafraîchissante.", 
    "La température de fermentation influence le goût final de la bière.", 
]

In [61]:
# get vocabulary
vocab = sorted(set([word for doc in corpus for word in doc.lower().split()]))

In [62]:
# compute posting list in a naïve whay
posting_list = {
    word: {
        f"doc-{doc_id}": [i for i, w in enumerate(doc_str.lower().split()) if w == word]
        for doc_id, doc_str in enumerate(corpus)
        if " " + word + " " in doc_str.lower()
    }
    for word in vocab
}

In [63]:
posting_list

{'ajoute': {'doc-3': [2]},
 'amertume': {'doc-3': [4]},
 'apporte': {'doc-4': [2, 12]},
 'arômes': {'doc-4': [14]},
 'au': {'doc-0': [4]},
 'basse': {'doc-5': [4]},
 'bière': {'doc-4': [7], 'doc-5': [1]},
 'bière.': {},
 'bières': {'doc-2': [5]},
 'blond': {'doc-0': [6]},
 'café': {'doc-4': [16]},
 'ce': {'doc-0': [1], 'doc-1': [2, 6]},
 'de': {'doc-4': [15], 'doc-5': [2], 'doc-6': [2, 8]},
 'des': {'doc-2': [4], 'doc-4': [13]},
 'distincte': {'doc-3': [5]},
 'donne': {'doc-2': [3]},
 'est': {'doc-5': [5]},
 'et': {'doc-5': [8]},
 'fermentation': {'doc-2': [1], 'doc-5': [3], 'doc-6': [3]},
 'final': {'doc-6': [7]},
 'fruitées.': {},
 'fume': {'doc-1': [1]},
 'goût': {'doc-6': [6]},
 'haute': {'doc-2': [2]},
 'houblon': {'doc-3': [1]},
 'influence': {'doc-6': [4]},
 'juge': {'doc-0': [5]},
 'la': {'doc-3': [7], 'doc-4': [3, 6, 18], 'doc-6': [0, 9]},
 'le': {'doc-6': [5]},
 'légère': {'doc-5': [7]},
 'mais': {'doc-4': [8]},
 'malt': {'doc-1': [3], 'doc-4': [1, 10]},
 'obtenir': {'doc-1':

In [64]:
posting_list["malt"] # appel à une hash-map => complexité en O(1) pour trouver les passages de tous les docs qui parlent d'un mot

{'doc-1': [3], 'doc-4': [1, 10]}

## BM25 : score de priorité pour un index lexical
Le score de référence dans le domaine de la recherche lexicale est le [BM25](https://www.wikiwand.com/fr/articles/Okapi_BM25) (pas nécessaire de comprendre les formules). En première approximation, supposons que ce score attribué à chaque document calcule, pour chaque mot de la query, la fréquence d'apparition du mot dans le document pondéré par des notions de rareté du mot dans le corpus (une `posting_list` permet d'obtenir cette information très facilement).



In [65]:
from bm25s import BM25, tokenize

# Tokenize the corpus and index it
corpus_tokens = tokenize(corpus)
retriever = BM25(corpus=corpus)
retriever.index(corpus_tokens)

retriever.retrieve(tokenize("malt"), k=3)

Results(documents=array([['Le malt apporte la rondeur à la bière mais un malt torréfié apporte des arômes de café à la bière.',
        'On fume ce malt pour obtenir ce whiskey tourbé',
        'La température de fermentation influence le goût final de la bière.']],
      dtype='<U98'), scores=array([[0.5355514, 0.5169559, 0.       ]], dtype=float32))

## Index Vespa
[Vespa](https://vespa.ai/) est un index moderne, puissant, hautement distribué, concurrent du très connu [ElasticSearch](https://www.elastic.co/fr/). Ces 2 index permettent d'être utilisés en mode lexical pour de la recherche d'information dans un corpus.

Nous utiliserons Vespa pendant ce TP : ne pas hésiter à [aller voir la doc](https://docs.vespa.ai/en/overview.html) (attention, ne pas se laisser impressionner par toute la technique) ou poser des questions à leur [chat-bot](https://search.vespa.ai/) (même remarque).

Une instance Vespa tourne sur le serveur à l'adresse http://vespa:8080 . Pour y accéder facilement nous utiliserons le package Python PyVespa (voir [le doc](https://pyvespa.readthedocs.io/en/latest/index.html)) proposé par l'équipe Vespa :

In [121]:
from vespa.application import Vespa
import json

vespa = Vespa(url="http://vespa", port=8080)
vespa.wait_for_application_up(30)

Application is up!


In [29]:
# requête simple
resp = vespa.query(
    {
        "yql": "select * from beer where userQuery()", # <-- on dirait du SQL !
        "hits": 2, # <-- nombre de résultats voulus
        "query": "stout", # <-- query textuelle, nous verrons son usage après
    }
)
print("Réponse de Vespa:\n")
print(json.dumps(resp.json["root"], indent=2))

Application is up!
Réponse de Vespa:

{
  "id": "toplevel",
  "relevance": 1.0,
  "fields": {
    "totalCount": 79
  },
  "coverage": {
    "coverage": 100,
    "documents": 959,
    "full": true,
    "nodes": 1,
    "results": 1,
    "resultsFull": 1
  },
  "children": [
    {
      "id": "id:beer:beer::beer:4134",
      "relevance": 3.7397587588627377,
      "source": "beer_content",
      "fields": {
        "sddocname": "beer",
        "documentid": "id:beer:beer::beer:4134",
        "name": "Kalamazoo Stout",
        "description_beer": "A full-bodied stout with plenty of roast flavor. Kalamazoo Stout is available year round, leading our vast portfolio of stouts.",
        "brewery": "Bell's Brewery Inc.",
        "address1": "8938 Krum Ave.",
        "country": "United States",
        "cat_name": "British Ale",
        "style_name": "Sweet Stout",
        "summaryfeatures": {
          "bm25(brewery)": 0.0,
          "bm25(description_beer)": 3.7397587588627377,
          "lengt

Un index (Vespa ou ES) a besoin de connaître le schéma de data à indexer. Il s'agit de :
- l'ensemble des attributs d'un document à indexer : nom, type (string, float, ...)
- si nécessaire, la façon dont un champ doit être utilisés par Vespa lors de l'indexation : simple attribut, indexé par `posting list`, exploitable via BM25, affichable dans les réponses aux queries, etc ...

Elastic nomme cela [mapping](https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping.html), Vespa nomme cela la [schema](https://docs.vespa.ai/en/schemas.html). Exemple avec nos données montées sur Vespa
```
schema beer {
    document beer {
        field id type string {
        }
        field name type string {
            indexing: index | summary
        }
        field description_beer type string {
            indexing: index | summary | attribute
            index: enable-bm25
        }
        field brewery type string {
            indexing: index | summary
            index: enable-bm25
        }
        field address1 type string {
            indexing: index | summary | attribute
        }
        field description_brewery type string {
            indexing: index
            index: enable-bm25
        }
    [...]
    }

  [...]

}
```

Vespa est très orienté recherche d'information et permet également de définir des façons de très précise le **score d'adéquation** à utiliser pour classer (ranker) les documents. Il s'agit d'une autre partie du schéma :
```
  [...]

    rank-profile basic_bm25 {
        first-phase {
            expression {
                bm25(name) + bm25(description_beer)
            }
        }
    }

  [...]
```

# Uses cases

## UC-1 : description data

Attention, plus compliqué qu'en SQL. Voir les docs spécifiques
- [doc spécifique Vespa sur le grouping](https://docs.vespa.ai/en/grouping.html)
- [doc sur le SQL façon Vespa - YQL](https://docs.vespa.ai/en/vespa-cli#cheat-sheet)

**Ne pas chercher à aller jusqu'au bout !!** Nous remarquerons assez vite que c'est galère avec Vespa

- Q1: Combien y a-t-il de bières dans la DB ?
- Q2: Top10 brasseries les plus représentées avec le nombre de bière par brasserie ?
- Q3: Top10 des bières les plus fortes (ABV) en France ?
- Q4: Par pays, nombre de brasseries qui proposent des bières de type `Porter` et ABV moyen de celles-ci ?
- Q5: Mediane du nombre de bière par pays ?

In [113]:
# your code

# UC-2 : préparer un dataset de ranking 

Plusieurs problèmes pour résoudre ce use-case:
- cf intro : un index ne peut pas faire de jointure 
- cf UC-1 : les grouping sont horribles à réaliser

Conclusion : les index ne sont vraiment pas adaptés pour fabriquer des datasets !

In [113]:
# your code

# UC-3 : récupérer les docs qui parlent d'un mot

Peut-on utiliser Vespa pour réaliser un mini moteur de recherche ? 

**Rappel:** une configuration Vespa intègre la définition d'un **score d'adéquation** entre query et document ! Le `rank-profile` suivant existe sur l'instance Vespa utilisées :
```
  [...]

    rank-profile rank-brewery-and-descr inherits root{
        first-phase {
            expression {
                bm25(description_brewery) + bm25(description_beer)
            }
        }
    }

  [...]
```

**Question 1:** expliquer en 2 phrases comment le rank-profile `rank-brewery-and-descr` va agir.

**Question 2:** pour différentes requêtes textuelles très simples à base de mot-clef, retrouver les bières qui semblent répondre à la demande :
- trouver les bières ou les brasseries qui parlent de bières "fine"
- idem pour "juicy"
- idem pour "genuine"
- idem pour les bières mâturées dans des "oak cask" (fûts en chêne) -> combien y en a-t-il ? $N_1$
   - idem pour les bières qui évoquent uniquement "cask" -> combien y en a-t-il ? $N_{1,1}$
   - idem pour celles ne parlant que de "oak" -> combien y en a-t-il ? $N_{1,2}$
- idem pour les bières qui évoquent "oak" et "cask" -> combien y en a-t-il ? $N_{2}$

In [113]:
# your code

# UC-4 : vectorisation des description des bières

## Index et "scrolling/visiting"

Les index ne sont "pas faits" pour manipuler la donnée qu'ils hébergent, néanmoins il existe plusieurs systèmes qui permettent à un code arbitraire de s'exécuter sur chaque document. Vespa nomme son système `visitor` ([doc](https://docs.vespa.ai/en/visiting.html)) et permet de 
- selectionner des documents à processer
- dumper l'index en parallèle (et possiblement les modifier + update à la volée)
- appliquer un code arbitraire à des documents (Nécessite le recours à Java pour expliciter l'opération :o )

L'équivalent Elastic Search est le [scroll](https://www.elastic.co/guide/en/elasticsearch/guide/current/scroll.html).

**Remarque:** on comprend vite qu'il n'est pas forcément simple de manipuler une donnée une fois qu'elle est indexée. Vespa étant très tourné vers le ML, il est toutefois possible de vectoriser des documents automatiquement lors de leur upsert.

# UC-5 : answer question in corpa

**impossible en SQL**

**Grandes lignes :** trouvons les documents qui répondent à une question. Exemple : à partir de la description vectorisée à UC-4 pour chaque bière, comment trouver les bières qui répondent à une description plus complète ? Exemple:
- "very bitter beer with smoky taste"
- "fruity sour - balanced sourness"
- "weird beer"


## Index vectoriel : NN et ANN, équivalent ML des index lexicaux
Depuis ~2020, le ML permet de représenter des documents textuels par des vecteurs en grande dimension (appelés **embedding**) qui possèdent l'énorme propriété de traduire numériquement/vectoriellement l'information sémantique contenue dans les documents. De surcroit, ces embeddings peuvent se comparer algébriquement très simplement dans le sens où 2 embeddings "proches" (dans leur espace) correspondent à des objets proches (dans notre perception). Le uses-case 4 des TP précédent visait à obtenir de tels embeddings à partir d'un service externe.

et les algorithmes *Nearest Neighbors* ou plus récemment *Approximated Nearest Neighbors*.
### Recherche par plus proche voisins 
#### Algorithme Nearest Neighbors - NN

Puisqu'il est possible de représenter (presque) tout document sous format embedding et que ces derniers ont la propriété d'être comparables entre eux, un nouveau type de recherche s'ouvre : recherche par embeddings les plus proche de l'embedding d'une query. Opérer une recherche à partir d'une query revient à trouver les *plus proches voisins* (Nearest Neighbors - NN) de l'embedding de la query parmi les embeddings de documents.

Exemple en dimension 2 : les coordonnées GPS 2D d'une ville sont en quelque sort un embedding basique d'une ville. Trouver les 5 villes les plus proches de Nancy est simple : il suffit de cacluler les distances de Nancy à toutes les villes grâce à leurs coordonnées et de trouver les top5 distances les plus faibles.

**Problème:** une telle recherche exhaustive implique $O(n^2)$ calculs où $n$ est le nombre de villes. Si $n=10^6$, le calcul devient difficile.

#### Variante Approximative Nearest Neihbors - ANN

**Solution proposée:** sachant qu'il est inutile de calculer la distance entre Nancy et Timbuktu ou New-York (celles-ci ne seront jamais dans le top5 proximité), il peut être intéressant de restreindre le champ de recherche afin de ne payer une recherche exhaustive en $O(n^2)$ que pour une poignée de villes qu'on sait déjà être "proches". Dans notre exemple, une recherche limitée au département de la ville cible et aux départements limitrophes suffit. 

Il s'agit d'un début de compréhension de la famille d'algorithmes Approximative Nearest Neihbors (ANN par la suite) qui permet de casser la complexité du problème de recherche de plus proches voisins en hierarchisant l'information. Cette hierarchisation se fait via une structure de donnée particulière ; l'implémentation la plus courante en 2024 est [Hierarchical Navigable Small World - HNSW](https://www.wikiwand.com/en/articles/HNSW_indexes).

Remarque : l'algo qui traduit réellement la hierarchisation stricte est plutôt de la famille [K-d tree](https://www.wikiwand.com/en/articles/K-d_tree) mais il est inefficace pour des vecteurs de dimension $k=768+$ comme c'est le cas pour la plupart des embeddings